# Projektarbeit – Hierarchisches PRM-Verfahren

## Technischer Rückblick auf den Projektverlauf

Im Rahmen unserer Projektarbeit zum hierarchischen PRM-Verfahren haben wir am **26.06.2025** die Aufgabenstellung erhalten. Ziel war es, ein funktionierendes hierarchisches Planungsverfahren auf Basis des PRM-Ansatzes zu entwickeln und anschließend hinsichtlich unterschiedlicher Freiheitsgrade zu evaluieren.

### Woche 1 – Theoretische Einarbeitung und Planungsphase

In der ersten Projektwoche lag der Fokus auf einer fundierten **theoretischen Einarbeitung**. Wir analysierten die zugrundeliegenden Prinzipien des klassischen PRM sowie die Erweiterung auf ein **hierarchisches Planungsverfahren**, bei dem ein Hauptplaner und mehrere Subplaner gemeinsam agieren. Gleichzeitig begannen wir mit der Grobplanung der Softwarearchitektur und möglichen Abstraktionsstrategien.

### Woche 2 – Erste Implementierungsansätze

In der zweiten Woche entwickelten wir erste **Implementierungen beider Planner-Ebenen**. Die initialen Versionen schienen funktional korrekt zu sein – insbesondere der Hauptplaner erzeugte konsistente und gültige Pfade. Eine detaillierte Interaktion zwischen Haupt- und Subplaner war zu diesem Zeitpunkt jedoch noch nicht vollständig getestet.

![Erste Visuelleisierung des Hierarchical PRM](./images/Fat_Bottleneck_HierLazy.png)

### Woche 3 – Fehleranalyse und externe Probleme

In Woche drei erfolgte eine umfassendere **Testphase**, in der wir feststellten, dass unser **Subplanner nicht wie vorgesehen arbeitete**. Die resultierenden Pfade des Subplanners entsprachen fast immer exakt denen des Hauptplaners – was auf eine mangelnde Entkopplung und Interaktion zwischen den Ebenen hindeutete. Der Subplanner wurde faktisch umgangen, da der globale Planner meist bereits den optimalen Pfad erzeugte.

Parallel dazu erlitten wir einen erheblichen Rückschlag: Durch das Pullen des aktuellen Stands aus dem GitHub-Repository wurde unsere Arbeitsumgebung **unbrauchbar**, da die externe Version fehlerhafte Komponenten enthielt. In dem Glauben, der Fehler läge bei uns, **verwarfen wir unseren funktionierenden Code vollständig** und begannen neu – was zu einem nicht zu unterschätzenden Zeitverlust führte.

![Hinzufügen eines nicht funktionierenden Github-Commits](./images/adding_latest_lecture_setup.png)

### Woche 4 – Neuentwicklung und finale Struktur

Daraufhin entschieden wir uns für einen vollständigen **Neustart der PRM-Implementierung**, diesmal mit einem stark modularisierten Aufbau und klaren Trennung der Verantwortlichkeiten. Diese Lösung funktionierte technisch einwandfrei und war deutlich stabiler als vorherige Ansätze.

![Erste funktionierende PRM-Implementierung mit Subplaner](./images/Hierarchical_PRM_Implementation.png)

Allerdings wichen wir durch den strukturellen Neuansatz in Teilen von der vorgegebenen Simulationsstruktur ab, was zur Folge hatte, dass **die Integration in die ursprünglich vorgesehene Visualisierung nicht mehr vollständig möglich war**.

![Eigene Animation von 2 DoF](./images/Simulation_2DoF.png)

### Einschränkungen bei der Evaluation

Ein zentrales Ziel der Projektaufgabe – die **Vergleichsstudie der Performance bei 4, 8 und 12 Freiheitsgraden** unter Anwendung unterschiedlicher PRM-Varianten – konnte aus diesem Grund **nicht in vollem Umfang umgesetzt werden**.

Unsere Implementierung ließ sich zwar teilweise ausführen, jedoch kam es bei der Ausführung im mitgelieferten Simulationsframework entweder zu **inkonsistenten Kollisionen innerhalb der Visualisierung**, zu **exorbitanten Rechenzeiten** oder sogar zu vollständigem **Abbruch durch Timeouts**. Diese Probleme verhinderten eine belastbare und vergleichbare Auswertung der geplanten Testfälle.

## Fazit

Trotz mehrfacher Rückschläge – sowohl auf konzeptioneller als auch technischer Ebene – gelang es uns, ein grundsätzlich funktionierendes hierarchisches PRM-System zu entwickeln. Die Projektarbeit zeigte deutlich, wie wichtig robuste Infrastruktur, funktionierende Referenzumgebungen und modulare Entwicklung bei komplexeren Planungsproblemen sind. Ein umfassender Test der Performance hinsichtlich verschiedener Freiheitsgrade wäre ein logischer nächster Schritt bei stabilerer Integrationsumgebung.


# Final Report – Hierarchical Motion Planning

## 1. Implementation of the Hierarchical Planner

### 1.1 Architecture

#### A. Global Planner: Visibility Roadmap

The global planner is based on a visibility graph. Nodes ("guards") are placed in free space, and edges are created only if there is a direct line of sight. This results in a lightweight and efficient roadmap for coarse planning in complex environments.

#### B. Local Planners: BasicPRM and LazyPRM

For each segment $(q_i, q_{i+1})$ of the global path, a new local PRM instance is created. Two variants were tested:

- **BasicPRM**: performs collision checks during roadmap construction
- **LazyPRM**: delays collision checks until path extraction

Local planning is performed within a bounding box around each segment:

$$
\text{local\_limits}_x = \left[ \max(c_x - p, x_{\min}),\ \min(c_x + p, x_{\max}) \right]
$$

$$
\text{local\_limits}_y = \left[ \max(c_y - p, y_{\min}),\ \min(c_y + p, y_{\max}) \right]
$$

---

### 1.2 Test Environments

- **2-DoF**: Used for validating the basic functionality of the system
- **4-DoF**: Revealed limitations due to fixed arm lengths and constrained reachability
- **Custom robot class**: Enabled faster simulations but was prone to invalid collision states

Conclusion: Realistic environments for 4+ DoF were unstable due to numerical and modeling limitations.

---

### 1.3 Complex Scenarios

Goal: Evaluate the hierarchical planner in five complex environments using robots with 4, 8, and 12 DoF.

#### 1.3.1 Benchmarking

Planned metrics:

- Path length:
  $$
  L = \sum_{i=0}^{N-1} \| q_{i+1} - q_i \|
  $$
- Number of nodes/edges
- Planning time
- Success rate per segment

Benchmarking was limited due to unreliable high-DoF simulations.

#### 1.3.2 Animation of Robot Motions

Planned visualization:

- Spline interpolation:
  $$
  \gamma(t) = \sum B_i(t) \cdot q_i
  $$
- 2D animation of configurations

Robust animations for realistic 6+ DoF scenarios could not be achieved reliably.

#### 1.3.3 Discussion: Solving Complex Problems

Despite technical constraints, the hierarchical planner is theoretically well suited for:

- Modular decomposition into subproblems
- Isolated debugging at the segment level
- Subplanner flexibility and exchangeability

---

## 2. Theoretical Classification and Path Use

### 2.1 Single vs. Multi-Query Methods

- **BasicPRM & LazyPRM** are designed as **multi-query** methods
- However, in the hierarchical structure, each segment uses a fresh PRM instance → **single-query** in practice

Thus, the planner represents a **hybrid system** combining a multi-query global structure with single-query subplanners per path segment.

---

### 2.2 Path Execution via Robot Commands

A planned path consists of discrete waypoints $q_i$, which could be executed via:

- `PTP q[i]` or `LIN q[i]` in systems like KUKA KRL or URScript

**However:** Such direct execution is not advisable, because:

- Paths are generated without velocity or acceleration limits
- No trajectory optimization is applied
- Segment transitions may be discontinuous

Hence, **direct execution is technically feasible, but not practically useful**, and may lead to jerky or unsafe motions.

---

### 2.3 Executing Paths Smoothly

To generate executable and safe robot motions, the following steps are required:

1. **Spline interpolation** of the waypoints:
   $$
   \gamma(t) = \sum B_i(t) q_i
   $$

2. **Velocity and acceleration limiting**:
   $$
   \|\dot{\gamma}(t)\| \leq v_{\max},\quad \|\ddot{\gamma}(t)\| \leq a_{\max}
   $$

3. **Time parameterization** across all joints

4. **Collision checking along the continuous path**, not just the sampled waypoints

These steps are essential to produce physically smooth and safe trajectories.

---

### 2.4 Improving Path Quality

To improve the resulting motion path, several techniques can be applied:

- **Trajectory smoothing** (e.g., CHOMP, STOMP)
- **Path length reduction** via post-processing
- **Adaptive resampling** in complex regions
- **Cost functions** (e.g., energy, time, clearance)

These techniques significantly enhance the feasibility and industrial relevance of the path.

---

## 3. Conclusion

The developed hierarchical planner demonstrates solid conceptual strengths:

- Clear separation of global and local planning
- Modular structure with exchangeable subplanners
- Segment-level diagnostics and flexibility

However, due to kinematic and numerical simulation limits, full validation was not possible. Nonetheless, the architecture provides a robust foundation for future robotic planning in realistic systems.

---

**Authors:** Simon Boes, Mert B.
**Date:** July 2025
**Course:** 24179 – Innovative Concepts for Industrial Robot Programming
**Instructor:** Prof. Dr. Björn Hein
**Institution:** Karlsruhe Institute of Technology (KIT)


# Abschlussbericht – Hierarchische Bewegungsplanung

## 1. Umsetzung des hierarchischen Planers

### 1.1 Implementierung

#### A. Hauptplaner: Visibility Roadmap

Der Hauptplaner basiert auf einem Sichtbarkeitsgraph. Knoten (Guards) werden in freiem Raum platziert, Kanten nur bei direkter Sichtverbindung erzeugt. So entsteht eine globale, effiziente Struktur zur Grobplanung.

#### B. Interner Planer: BasicPRM und LazyPRM

Für jedes Segment $(q_i, q_{i+1})$ wird lokal eine neue PRM-Instanz aufgebaut. Zwei Varianten wurden getestet:

- **BasicPRM**: mit sofortiger Kollisionserkennung beim Aufbau
- **LazyPRM**: mit verzögerter Kollisionserkennung bei Pfadnutzung

Der lokale Planer arbeitet in einem begrenzten Raum (Bounding Box) um das Segment:

$$
\text{local\_limits}_x = \left[ \max(c_x - p, x_{\min}),\ \min(c_x + p, x_{\max}) \right]
$$

$$
\text{local\_limits}_y = \left[ \max(c_y - p, y_{\min}),\ \min(c_y + p, y_{\max}) \right]
$$

---

### 1.2 Testumgebungen

- **2-DoF**: Zur Validierung grundlegender Funktion
- **4-DoF**: Zeigte Einschränkungen durch feste Armlängen
- **Eigene Roboterklasse**: Schneller, aber fehlerhaft (Kollisionen wurden ignoriert)

Ergebnis: Realistische Testumgebungen mit mehr als 4 DoF waren aufgrund numerischer Instabilität nicht zuverlässig nutzbar.

---

### 1.3 Komplexe Szenarien

Ziel: Planung in 5 komplexen Umgebungen mit 4, 8 und 12 DoF

#### 1.3.1 Benchmarking

Geplante Metriken:

- Pfadlänge:
  $$
  L = \sum_{i=0}^{N-1} \| q_{i+1} - q_i \|
  $$
- Anzahl Knoten/Kanten
- Planungszeit
- Erfolgsrate je Segment

Vergleichbarkeit war begrenzt, da komplexe Roboterkonfigurationen nicht sicher simulierbar waren.

#### 1.3.2 Animationen

Geplante Darstellung über:

- Spline-Interpolation:
  $$
  \gamma(t) = \sum B_i(t) \cdot q_i
  $$
- Animierte Visualisierung im 2D-Raum

Robuste Animationen für realistische 6+ DoF Modelle waren nicht möglich.

#### 1.3.3 Diskussion komplexer Aufgaben

Trotz technischer Limitierungen ist der hierarchische Planer theoretisch gut geeignet für:

- Modularisierung in Subaufgaben
- Fehlerdiagnose auf Segmentebene
- Austauschbarkeit der Subplaner

---

## 2. Theoretische Einordnung und Pfadverwendung

### 2.1 Single- vs. Multi-Query

- **BasicPRM & LazyPRM**: konzipiert als Multi-Query-Verfahren
- **Im hierarchischen Aufbau** jedoch: lokale Planer arbeiten wie Single-Query-Instanzen, da sie pro Segment neu erzeugt werden

Insgesamt handelt es sich beim hierarchischen Planer um eine **hybride Struktur**, bei der ein Multi-Query-Hauptplaner mit segmentbasierten Single-Query-Komponenten kombiniert wird.

---

### 2.2 Bewegungsbefehle für Roboterprogramme

Ein Ergebnispfad besteht aus diskreten Wegpunkten $q_i$. Diese könnten direkt mit standardisierten Bewegungsbefehlen wie `PTP q[i]` oder `LIN q[i]` ausgeführt werden – z. B. in KUKA KRL oder URScript.

**Aber:** Die Pfade wurden:

- ohne Dynamikgrenzen (v, a) geplant
- ohne Trajektorienoptimierung erzeugt
- mit potenziellen Kantendiskontinuitäten zwischen Segmenten

Daher ist die **direkte Ausführung zwar möglich, aber nicht sinnvoll**, da sie zu ruckhaften Bewegungen, Kollisionen oder ineffizienter Trajektorienführung führen kann.

---

### 2.3 Dynamisch sinnvolle Ausführung

Um einen geplanten Pfad sinnvoll und flüssig abzufahren, sind folgende Schritte nötig:

1. **Interpolation** der diskreten Wegpunkte mittels Splines:
   $$
   \gamma(t) = \sum B_i(t) q_i
   $$

2. **Geschwindigkeits- und Beschleunigungsbegrenzung**:
   $$
   \|\dot{\gamma}(t)\| \leq v_{\max},\quad \|\ddot{\gamma}(t)\| \leq a_{\max}
   $$

3. **Zeitparametrierung** zur Synchronisation aller Gelenke

4. **Kollisionstest entlang des interpolierten Pfads**, nicht nur an den diskreten Punkten

Dies führt zu physikalisch konsistenteren, weicheren und sichereren Bewegungen.

---

### 2.4 Verbesserung der Pfadqualität

Zur Optimierung des Ergebnispfades bieten sich folgende Methoden an:

- **Trajektorienglättung** (z. B. CHOMP, STOMP)
- **Reduktion der Pfadlänge** durch Post-Processing
- **Segmentweises Neusampling** mit adaptiver Dichte
- **Kostenfunktionen** zur Optimierung bzgl. Energie, Zeit oder Sicherheit

Diese Schritte verbessern sowohl die Ausführbarkeit als auch die industrielle Nutzbarkeit erheblich.

---

## 3. Fazit

Der entwickelte hierarchische Planer zeigt konzeptionell starke Ansätze:

- Trennung von globaler und lokaler Planung
- Austauschbare Subplaner
- Strukturierte Fehleranalyse auf Segmentebene

In der Praxis verhinderten jedoch numerische und kinematische Limitationen eine vollständige Evaluation. Für reale Robotersysteme oder verbesserte Simulationsinfrastrukturen bietet das System jedoch eine robuste, modulare Basis.

---

**Autoren:** Simon Boes, Mert B.
**Datum:** Juli 2025
**Vorlesung:** 24179 – Innovative Konzepte zur Programmierung von Industrierobotern
**Professor:** Prof. Dr. Björn Hein
**Institution:** Karlsruher Institut für Technologie (KIT)
